In [1]:
import pandas as pd, numpy as np
import vivarium_output_loader as vol
import lsff_output_processing as lop
# import lsff_summarizer
# from lsff_summarizer import LSFFOutputSummarizer

!whoami
!date

ndbs
Sat Apr 18 22:41:53 PDT 2020


In [20]:
%load_ext autoreload
%autoreload 2

## 1. Load count data from all locations

In [2]:
base_directory3 = '/share/costeffectiveness/results/vivarium_conic_lsff/v3.0_paf_of_one'


locations_rundates3 = {
    'Ethiopia': '2020_03_17_14_59_54',
    'India': '2020_03_17_15_00_02',
    'Nigeria': '2020_03_17_15_00_10',
}

In [21]:
data = vol.load_transformed_count_data_and_merge_locations(base_directory3, locations_rundates3)
data.keys()

dict_keys(['gestational_age', 'transition_count', 'deaths', 'state_person_time', 'births_with_ntd', 'population', 'person_time', 'ylls', 'ylds', 'births', 'birth_weight'])

## 2a. Check person time data tables

In [22]:
data['person_time'].head()

,location,year,age_group,sex,vitamin_a_cat,anemia_group,measure,input_draw,value
0,Ethiopia,2020,1_to_4,female,cat1,mild,person_time,21,44.907203
1,Ethiopia,2020,1_to_4,female,cat1,mild,person_time,29,50.865774
2,Ethiopia,2020,1_to_4,female,cat1,mild,person_time,55,55.073370
3,Ethiopia,2020,1_to_4,female,cat1,mild,person_time,78,43.449582
4,Ethiopia,2020,1_to_4,female,cat1,mild,person_time,155,44.586546


In [23]:
data['person_time'].shape

(24000, 9)

In [24]:
data['state_person_time'].head()

,location,year,age_group,sex,cause,vitamin_a_cat,anemia_group,measure,input_draw,value
0,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,person_time,21,187.589322
1,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,person_time,29,194.436687
2,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,person_time,55,216.386037
3,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,person_time,78,161.347023
4,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,person_time,155,180.558522


In [25]:
data['state_person_time'].shape

(144000, 10)

### Hmm, no iron deficiency or vitamin A deficiency in the causes for state person time

So it looks like to get prevalence for either of these, we would group by the appropriate columns (`vitamin_a_cat` or `anemia_group`) in the person_time table.

### Also no person-time for neural tube defects, so we can't compute NTD prevalence except at birth

In [26]:
data['state_person_time'].cause.unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'susceptible_to_diarrheal_diseases',
       'susceptible_to_lower_respiratory_infections',
       'susceptible_to_measles'], dtype=object)

In [54]:
data['person_time'].input_draw.nunique()

25

## 2b. Compute anemia prevalence by age group and severity

In [55]:
# Compute anemia prevalence by age group and severity
lop.set_global_index_columns(['location', 'input_draw'])
numerator = data['person_time']
denominator = data['person_time']
numerator_strata = ['age_group', 'anemia_group']
denominator_strata = ['age_group']
anemia_prevalence = lop.rate_or_ratio(numerator, denominator, numerator_strata, denominator_strata)
anemia_prevalence.head()

,age_group,location,input_draw,anemia_group,value
0,1_to_4,Ethiopia,21,mild,0.268931
1,1_to_4,Ethiopia,21,moderate,0.321483
2,1_to_4,Ethiopia,21,none,0.387533
3,1_to_4,Ethiopia,21,severe,0.022052
4,1_to_4,Ethiopia,29,mild,0.272260


In [56]:
anemia_prevalence.groupby(['location'] + numerator_strata).value.describe()

count      mean       std       min  \
location age_group      anemia_group                                        
Ethiopia 1_to_4         mild           25.0  0.269805  0.004267  0.260869   
                        moderate       25.0  0.313601  0.009044  0.295009   
                        none           25.0  0.396523  0.012845  0.375574   
                        severe         25.0  0.020071  0.001085  0.018280   
         early_neonatal mild           25.0  0.267002  0.004114  0.258732   
                        moderate       25.0  0.339953  0.007161  0.324257   
                        none           25.0  0.369195  0.010592  0.352299   
                        severe         25.0  0.023851  0.000885  0.022469   
         late_neonatal  mild           25.0  0.268353  0.004119  0.259830   
                        moderate       25.0  0.334735  0.007340  0.319009   
                        none           25.0  0.373705  0.010820  0.356330   
                        severe         25.0  0.023207  0.000901  0.021933   
         post_neonatal  mild           25.0  0.269476  0.004168  0.260636   
                        moderate       25.0  0.319600  0.008426  0.302097   
                        none           25.0  0.389990  0.012094  0.370635   
                        severe         25.0  0.020933  0.000974  0.019448   
India    1_to_4         mild           25.0  0.287813  0.003071  0.280684   
                        moderate       25.0  0.286143  0.004631  0.277884   
                        none           25.0  0.413501  0.007081  0.398801   
                        severe         25.0  0.012544  0.000375  0.011952   
         early_neonatal mild           25.0  0.285384  0.002293  0.279968   
                        moderate       25.0  0.301407  0.003929  0.295211   
                        none           25.0  0.397441  0.005746  0.386927   
                        severe         25.0  0.015767  0.000308  0.015314   
         late_neonatal  mild           25.0  0.287187  0.002329  0.281718   
                        moderate       25.0  0.296910  0.004035  0.290639   
                        none           25.0  0.401983  0.005873  0.391202   
                        severe         25.0  0.013921  0.000319  0.013437   
         post_neonatal  mild           25.0  0.287668  0.002633  0.281399   
                        moderate       25.0  0.288859  0.004470  0.281001   
                        none           25.0  0.410558  0.006591  0.397160   
                        severe         25.0  0.012915  0.000329  0.012356   
Nigeria  1_to_4         mild           25.0  0.254993  0.008715  0.233988   
                        moderate       25.0  0.360173  0.047450  0.290161   
                        none           25.0  0.353848  0.054338  0.220139   
                        severe         25.0  0.030987  0.007925  0.020893   
         early_neonatal mild           25.0  0.252265  0.007669  0.238317   
                        moderate       25.0  0.378519  0.039893  0.323123   
                        none           25.0  0.334729  0.045726  0.220111   
                        severe         25.0  0.034488  0.006820  0.026066   
         late_neonatal  mild           25.0  0.254161  0.008018  0.238717   
                        moderate       25.0  0.373087  0.040839  0.316599   
                        none           25.0  0.339381  0.046709  0.222117   
                        severe         25.0  0.033370  0.007093  0.024428   
         post_neonatal  mild           25.0  0.254874  0.008423  0.236152   
                        moderate       25.0  0.363965  0.045239  0.299865   
                        none           25.0  0.349501  0.051779  0.220995   
                        severe         25.0  0.031660  0.007633  0.022527   

                                           25%       50%       75%       max  
location age_group      anemia_group                                          
Ethiopia 1_to_4         mild 

## 3. See if I can reproduce the nonsensical YLD results seen in `output.hdf`

Yes, looks like the same data (see `2020_04_17a_compare_model_output_files.ipynb`): There are YLDs due to vitamin A deficiency in groups where `vitamin_a_cat == 'cat2'`, which is supposed to be the *absence* of vitamin A deficiency.

In [27]:
ylds = data['ylds']
print(ylds.shape)
print(ylds.location.unique())
print(ylds.cause.unique()) # No ylds from measles, LRI, or NTDs?
print(ylds.vitamin_a_cat.unique())
print(ylds.anemia_group.unique())
ylds.head()

(72000, 10)
['Ethiopia' 'India' 'Nigeria']
['diarrheal_diseases' 'iron_deficiency' 'vitamin_a_deficiency']
['cat1' 'cat2']
['mild' 'moderate' 'none' 'severe']


,location,year,age_group,sex,cause,vitamin_a_cat,anemia_group,measure,input_draw,value
0,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,ylds,21,22.491460
1,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,ylds,29,26.883470
2,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,ylds,55,30.886019
3,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,ylds,78,26.885465
4,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,ylds,155,20.254785


In [28]:
vad_ylds = ylds.query('cause == "vitamin_a_deficiency"')
vad_ylds = vad_ylds.query('location == "Nigeria" and age_group == "1_to_4"')
vad_ylds = vad_ylds.query('vitamin_a_cat == "cat2" and anemia_group == "none"')
vad_ylds.head()

,location,year,age_group,sex,cause,vitamin_a_cat,anemia_group,measure,input_draw,value
550,Nigeria,2020,1_to_4,female,vitamin_a_deficiency,cat2,none,ylds,21,138.145595
551,Nigeria,2020,1_to_4,female,vitamin_a_deficiency,cat2,none,ylds,29,159.216477
552,Nigeria,2020,1_to_4,female,vitamin_a_deficiency,cat2,none,ylds,55,208.218167
553,Nigeria,2020,1_to_4,female,vitamin_a_deficiency,cat2,none,ylds,78,200.973797
554,Nigeria,2020,1_to_4,female,vitamin_a_deficiency,cat2,none,ylds,155,116.013360


In [29]:
vad_ylds.groupby('year').value.describe()

,count,mean,std,min,25%,50%,75%,max
year,,,,,,,,
2020,200.0,42.463187,62.359793,0.0,0.002478,4.508588,49.359592,235.964744
2021,200.0,85.822972,126.026280,0.0,0.004784,9.308261,101.039733,474.811778
2022,200.0,87.240319,128.471700,0.0,0.005082,9.239089,101.658594,482.982637
2023,200.0,88.341364,130.456522,0.0,0.005102,9.137019,101.741493,489.342824
2024,200.0,89.328181,132.076935,0.0,0.004895,9.293067,102.950906,493.802670


In [34]:
vad_ylds = ylds.query('cause == "vitamin_a_deficiency"')
vad_ylds = vad_ylds.query('location == "Nigeria"')
vad_ylds.groupby(['age_group', 'vitamin_a_cat']).value.describe()

count        mean         std       min  \
age_group      vitamin_a_cat                                             
1_to_4         cat1           1000.0   62.340274   48.359677  0.853347   
               cat2           1000.0  203.939749  155.913271  2.817547   
early_neonatal cat1           1000.0    0.424042    0.666585  0.000000   
               cat2           1000.0    1.359162    1.943330  0.000000   
late_neonatal  cat1           1000.0    1.092235    1.748611  0.000142   
               cat2           1000.0    3.583100    5.254214  0.000380   
post_neonatal  cat1           1000.0   13.248330   11.249942  0.232236   
               cat2           1000.0   48.957556   39.181298  0.861743   

                                    25%         50%         75%         max  
age_group      vitamin_a_cat                                                 
1_to_4         cat1           22.015973   55.775357   92.521236  295.716155  
               cat2           70.968186  192.375015  298.596032  976.537380  
early_neonatal cat1            0.016289    0.119069    0.555120    4.393167  
               cat2            0.047710    0.447054    1.838249    8.774532  
late_neonatal  cat1            0.017635    0.287773    1.459459   11.583601  
               cat2            0.065312    1.067460    4.579388   23.844569  
post_neonatal  cat1            4.480372   10.943939   17.923385   72.124187  
               cat2           19.071784   41.524509   66.134132  205.561598

## 4. Along similar lines, let's check if there are iron deficiency YLDs in groups with no anemia

Well, this is looking better than VAD did

In [30]:
iron_ylds = ylds.query('cause == "iron_deficiency"')
iron_ylds = iron_ylds.query('location == "Nigeria" and age_group == "1_to_4"')
iron_ylds = iron_ylds.query('vitamin_a_cat == "cat2" and anemia_group == "none"')
iron_ylds.head()

,location,year,age_group,sex,cause,vitamin_a_cat,anemia_group,measure,input_draw,value
350,Nigeria,2020,1_to_4,female,iron_deficiency,cat2,none,ylds,21,0.0
351,Nigeria,2020,1_to_4,female,iron_deficiency,cat2,none,ylds,29,0.0
352,Nigeria,2020,1_to_4,female,iron_deficiency,cat2,none,ylds,55,0.0
353,Nigeria,2020,1_to_4,female,iron_deficiency,cat2,none,ylds,78,0.0
354,Nigeria,2020,1_to_4,female,iron_deficiency,cat2,none,ylds,155,0.0


In [31]:
iron_ylds.groupby('year').value.describe()

,count,mean,std,min,25%,50%,75%,max
year,,,,,,,,
2020,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
iron_ylds = ylds.query('cause == "iron_deficiency"')
iron_ylds = iron_ylds.query('location == "Nigeria"')
iron_ylds.groupby(['age_group', 'anemia_group']).value.describe()

count         mean          std         min  \
age_group      anemia_group                                                
1_to_4         mild          500.0    69.127099    52.444650    5.727482   
               moderate      500.0  1468.190867  1045.184800  145.539985   
               none          500.0     0.000000     0.000000    0.000000   
               severe        500.0   362.339948   300.796317   26.025761   
early_neonatal mild          500.0     0.138844     0.175956    0.002334   
               moderate      500.0    16.903300    11.275381    2.290497   
               none          500.0     0.000000     0.000000    0.000000   
               severe        500.0     6.718936     6.020507    0.304670   
late_neonatal  mild          500.0     0.227453     0.274871    0.001861   
               moderate      500.0    50.494263    33.437127    6.612109   
               none          500.0     0.000000     0.000000    0.000000   
               severe        500.0    25.845915    21.922853    2.003599   
post_neonatal  mild          500.0    16.784349    13.566846    1.168315   
               moderate      500.0   452.095286   329.767876   54.583609   
               none          500.0     0.000000     0.000000    0.000000   
               severe        500.0   136.452830   128.308836   11.905359   

                                    25%          50%          75%          max  
age_group      anemia_group                                                     
1_to_4         mild           28.092448    50.974109    93.523514   231.450355  
               moderate      642.869854  1049.934377  2090.082759  4671.102718  
               none            0.000000     0.000000     0.000000     0.000000  
               severe        151.326757   256.788035   522.251760  1744.349289  
early_neonatal mild            0.030091     0.074579     0.167964     1.076538  
               moderate        7.605912    12.763095    27.017585    45.222315  
               none            0.000000     0.000000     0.000000     0.000000  
               severe          2.191011     4.485018     9.170326    28.253149  
late_neonatal  mild            0.056483     0.132643     0.299213     1.803908  
               moderate       22.993841    37.275918    82.042489   127.641712  
               none            0.000000     0.000000     0.000000     0.000000  
               severe          9.678498    19.234560    36.307119   115.936920  
post_neonatal  mild            6.287873    11.544314    23.146802    55.768453  
               moderate      195.169518   321.132597   687.042179  1448.351642  
               none            0.000000     0.000000     0.000000     0.000000  
               severe         51.149410    94.493264   192.388521   895.712117

## 5. Check more data tables

In [35]:
data.keys()

dict_keys(['gestational_age', 'transition_count', 'deaths', 'state_person_time', 'births_with_ntd', 'population', 'person_time', 'ylls', 'ylds', 'births', 'birth_weight'])

### Deaths and YLLs

In [37]:
data['deaths'].head()

,location,year,age_group,sex,cause,vitamin_a_cat,anemia_group,measure,input_draw,value
0,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,death,21,5.0
1,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,death,29,8.0
2,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,death,55,3.0
3,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,death,78,9.0
4,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,death,155,3.0


In [38]:
data['ylls'].head()

,location,year,age_group,sex,cause,vitamin_a_cat,anemia_group,measure,input_draw,value
0,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,ylls,21,424.232752
1,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,ylls,29,683.695536
2,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,ylls,55,253.604276
3,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,ylls,78,763.007286
4,Ethiopia,2020,1_to_4,female,diarrheal_diseases,cat1,mild,ylls,155,256.881070


### Births and births with NTDs

In [40]:
data['births'].head()

,location,year,sex,measure,input_draw,value
0,Ethiopia,2020,female,live_births,21,21508.0
1,Ethiopia,2020,female,live_births,29,21827.0
2,Ethiopia,2020,female,live_births,55,21629.0
3,Ethiopia,2020,female,live_births,78,21920.0
4,Ethiopia,2020,female,live_births,155,21866.0


In [41]:
data['births_with_ntd'].head()

,location,year,sex,measure,input_draw,value
0,Ethiopia,2020,female,live_births_with_ntds,21,42.0
1,Ethiopia,2020,female,live_births_with_ntds,29,38.0
2,Ethiopia,2020,female,live_births_with_ntds,55,27.0
3,Ethiopia,2020,female,live_births_with_ntds,78,41.0
4,Ethiopia,2020,female,live_births_with_ntds,155,39.0


### Transition count and population

In [42]:
data['transition_count'].head()

,location,year,age_group,sex,vitamin_a_cat,anemia_group,measure,input_draw,value
0,Ethiopia,2020,1_to_4,female,cat1,mild,diarrheal_diseases_to_susceptible_to_diarrheal...,21,12500.0
1,Ethiopia,2020,1_to_4,female,cat1,mild,diarrheal_diseases_to_susceptible_to_diarrheal...,29,11658.0
2,Ethiopia,2020,1_to_4,female,cat1,mild,diarrheal_diseases_to_susceptible_to_diarrheal...,55,12137.0
3,Ethiopia,2020,1_to_4,female,cat1,mild,diarrheal_diseases_to_susceptible_to_diarrheal...,78,9662.0
4,Ethiopia,2020,1_to_4,female,cat1,mild,diarrheal_diseases_to_susceptible_to_diarrheal...,155,11134.0


In [44]:
data['transition_count'].measure.unique()

array(['diarrheal_diseases_to_susceptible_to_diarrheal_diseases_event_count',
       'lower_respiratory_infections_to_susceptible_to_lower_respiratory_infections_event_count',
       'measles_to_susceptible_to_measles_event_count',
       'susceptible_to_diarrheal_diseases_to_diarrheal_diseases_event_count',
       'susceptible_to_lower_respiratory_infections_to_lower_respiratory_infections_event_count',
       'susceptible_to_measles_to_measles_event_count'], dtype=object)

In [43]:
data['population'].head()

,location,measure,input_draw,value
0,Ethiopia,total_population,21,846725.0
1,Ethiopia,total_population,29,847552.0
2,Ethiopia,total_population,55,846172.0
3,Ethiopia,total_population,78,847337.0
4,Ethiopia,total_population,155,846910.0


In [45]:
data['population'].measure.unique()

array(['total_population', 'total_population_dead',
       'total_population_living', 'total_population_tracked',
       'total_population_untracked'], dtype=object)

### Birth weight and gestational age

In [47]:
data['birth_weight'].head()

,location,measure,input_draw,value
0,Ethiopia,birth_weight_mean,21,3361.542087
1,Ethiopia,birth_weight_mean,29,3290.398268
2,Ethiopia,birth_weight_mean,55,3354.101350
3,Ethiopia,birth_weight_mean,78,3352.374550
4,Ethiopia,birth_weight_mean,155,3379.348886


In [48]:
data['birth_weight'].measure.unique()

array(['birth_weight_mean', 'birth_weight_sd'], dtype=object)

In [49]:
data['gestational_age'].head()

,location,measure,input_draw,value
0,Ethiopia,gestational_age_mean,21,38.715850
1,Ethiopia,gestational_age_mean,29,38.482497
2,Ethiopia,gestational_age_mean,55,38.755840
3,Ethiopia,gestational_age_mean,78,38.712993
4,Ethiopia,gestational_age_mean,155,38.631846


In [50]:
data['gestational_age'].measure.unique()

array(['gestational_age_mean', 'gestational_age_sd'], dtype=object)